In [2]:
from create_model import *
#importing functions from .py file

In [3]:
directory = "/Users/justycewatson/Desktop/CIERA_files/NGC2682/jw_output"
#computer directory the files will come from

model_cluster_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
model_cluster_statistic

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,605002016872204416,1.361236,0.557799,0.967022,0.658567,14.214572,0.000000,0.017193,0.213815,3.038420e-250,9475.632108
1,604942879467199360,1.352529,0.557799,0.967022,0.610061,15.058561,0.000000,0.019429,0.166502,1.142975e-147,10231.678292
2,604969031523465728,0.387865,0.557799,0.967022,0.263017,36.229304,0.000000,0.012712,0.148434,3.486940e-120,9596.183794
3,604921679508385664,1.676418,0.557799,0.967022,0.744099,12.044454,0.000001,0.008560,0.136923,1.933852e-101,9597.816566
4,604968958508607360,1.262185,0.557799,0.967022,0.600071,15.298958,0.000000,0.014429,0.142905,7.963583e-108,9969.498500
...,...,...,...,...,...,...,...,...,...,...,...
1423,604994045412975744,0.770949,0.557799,0.967022,0.473367,19.784625,0.000000,0.009089,0.167184,4.178415e-150,9861.275938
1424,604921924322164608,0.167193,0.557799,0.967022,0.091203,105.613775,0.000000,0.012850,0.154449,4.685580e-128,10044.534580
1425,604970062315630336,1.360707,0.557799,0.967022,0.639646,14.251756,0.000000,0.013919,0.147220,1.930928e-114,9815.960047
1426,598962292125778560,1.532101,0.557799,0.967022,0.686447,13.055923,0.000000,0.020750,0.124527,1.440514e-82,9909.201340


# Creating ML model

In [4]:
##reading in dataframe containing known sampling quality to train model on

df1 = pd.read_csv('/Users/justycewatson/Desktop/CIERA_files/NGC2682/CSV_Files/NGC2682_Age_Stats.csv',sep=',')
sampling_df = df1[df1['Single Sampling'].isna() == False]

In [5]:
X, y, clf, X_test, X_train, y_test = create_model(model_cluster_statistic, sampling_df)
y_pred = make_preds(X_test, clf, y_test = y_test)

Accuracy: 0.969626168224299
              precision    recall  f1-score   support

         Bad       0.99      0.97      0.98       359
        Good       0.88      0.94      0.91        69

    accuracy                           0.97       428
   macro avg       0.93      0.96      0.95       428
weighted avg       0.97      0.97      0.97       428

Feature Importance Ranking:
Width          0.347775
SnR            0.213674
Stdev          0.169446
KS_value       0.159941
Dip_value      0.071619
ESS            0.027710
Dip_p          0.008100
KS_p           0.001736
Upper_bound    0.000000
Lower_bound    0.000000
dtype: float64


/Users/justycewatson/Desktop/CIERA_files/BASE9_ML/create_model.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampling_df['source_id'] = sampling_df['source_id'].astype(str)


# Applying new dataset to saved model

In [6]:
directory = '/Users/justycewatson/Desktop/CIERA_files/NGC6819/ngc6819_single_resfiles' 

ngc6819_statistic = create_stats(directory)
# creating the data table using 'create_stats' function with the directory files
 
ngc6819_statistic

#need to send model same columns in xtest 
#convert columns from ngc6819 to np array -- send to make preds function

,source_id,Width,Upper_bound,Lower_bound,Stdev,SnR,Dip_p,Dip_value,KS_value,KS_p,ESS
0,gaia_2076279137552154112_sin2,1.611105,0.281932,0.241271,0.709164,12.619723,0.0,0.015125,0.157928,3.597347e-137,9879.884524
1,gaia_2076298933067389056_sin2,0.851659,0.281932,0.241271,0.492672,18.154950,0.0,0.075300,0.251903,0.000000e+00,9594.089383
2,gaia_2076584355102288128_sin2,1.568087,0.281932,0.241271,0.706918,12.674155,0.0,0.015795,0.167206,2.280672e-151,10148.754993
3,gaia_2076488525792609792_sin2,0.384405,0.281932,0.241271,0.494811,18.364221,0.0,0.052020,0.276816,0.000000e+00,10262.489990
4,gaia_2076581851147039232_sin2,1.684027,0.281932,0.241271,0.748072,11.787783,0.0,0.021270,0.180972,1.965286e-177,10315.159905
...,...,...,...,...,...,...,...,...,...,...,...
1683,gaia_2076393834663325568_sin2,1.420736,0.281932,0.241271,0.685752,12.768550,0.0,0.009732,0.169243,1.352753e-157,9803.841960
1684,gaia_2076380400005524224_sin2,1.370467,0.281932,0.241271,0.686048,12.883365,0.0,0.021400,0.139514,2.575757e-105,9359.235534
1685,gaia_2076299895139969536_sin2,0.444381,0.281932,0.241271,0.175729,54.419198,0.0,0.085369,0.160964,2.239363e-141,9884.121496
1686,gaia_2076393662864610688_sin2,1.519873,0.281932,0.241271,0.696125,12.398271,0.0,0.016644,0.158247,1.000989e-137,9820.977658


In [7]:
ngc6819_clean, ngc6819_array = prepare_df_formodel(ngc6819_statistic)
ngc6819_array

array([[1.61110472e+000, 2.81932000e-001, 2.41271000e-001, ...,
        1.57928287e-001, 3.59734719e-137, 9.87988452e+003],
       [8.51659000e-001, 2.81932000e-001, 2.41271000e-001, ...,
        2.51902834e-001, 0.00000000e+000, 9.59408938e+003],
       [1.56808700e+000, 2.81932000e-001, 2.41271000e-001, ...,
        1.67206478e-001, 2.28067240e-151, 1.01487550e+004],
       ...,
       [4.44381000e-001, 2.81932000e-001, 2.41271000e-001, ...,
        1.60963855e-001, 2.23936299e-141, 9.88412150e+003],
       [1.51987300e+000, 2.81932000e-001, 2.41271000e-001, ...,
        1.58247012e-001, 1.00098941e-137, 9.82097766e+003],
       [5.23203000e-001, 2.81932000e-001, 2.41271000e-001, ...,
        9.77510040e-002, 3.15694497e-052, 9.51192880e+003]])

In [8]:
y_pred_6819 = make_preds(ngc6819_array, clf)
y_pred_6819

array(['Bad', 'Bad', 'Bad', ..., 'Bad', 'Bad', 'Bad'], dtype=object)

# Comparing ML labels with Dr. Jeffrey's labels

In [10]:
column_names = ['file', 'star', 'rank']
txt_6819 = pd.read_csv( '/Users/justycewatson/Desktop/CIERA_files/NGC6819/ngc6819_checkRes_all.txt',sep='\s+',names=column_names, skiprows=1)
txt_6819=txt_6819.drop('file', axis=1)

txt_6819

# rank 1 - good sampling (chain converges reasonably)
# rank 2 - poor sampling (no sampling or highly correlated)
# rank 3 - parts of chain seem convergent
# rank 4 - flat distribution 

,star,rank
0,2076220554197701504,4
1,2076220829075672960,4
2,2076220966514688128,3
3,2076224784750223232,4
4,2076227116907912960,4
...,...,...
1683,2076616859415037824,3
1684,2076617276038017664,4
1685,2076617447828109440,4
1686,2076617477890443008,2
